In [5]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import albumentations as A # 데이터 증식

In [6]:
transform = A.Compose([
    A.HorizontalFlip(p=0.5), # 좌우 반전
    A.RandomBrightnessContrast(brightness_limit=0.3, contrast_limit=0.3, p=0.5), # 밝기 및 대비 조절
    A.ShiftScaleRotate(p=0.5), # 이동, 확대/축소, 회전
])

data = []
labels = []

# Clean 폴더 이미지 불러오기
clean_dir = 'Data_PlasticCup/clean'
for filename in os.listdir(clean_dir):
    img = cv2.imread(os.path.join(clean_dir, filename))
    img = cv2.resize(img, (128, 128))
    augmented = transform(image=img)
    augmented_img = augmented['image']
    data.append(augmented_img)
    labels.append(0)  # 0 = 깨끗한 컵

# NG 폴더 이미지 불러오기
ng_dir = 'Data_PlasticCup/ng'
for filename in os.listdir(ng_dir):
    img = cv2.imread(os.path.join(ng_dir, filename))
    img = cv2.resize(img, (128, 128))
    augmented = transform(image=img)
    augmented_img = augmented['image']
    data.append(augmented_img)
    labels.append(1)  # 1 = 더러운 컵

# 데이터 분할
x_train, x_val, y_train, y_val = train_test_split(data, labels, test_size=0.2, random_state=42)

In [7]:
# 모델 정의
model = Sequential()

# 컨볼루션 레이어 1
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)))
model.add(MaxPooling2D((2, 2)))

# 컨볼루션 레이어 2 
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

# 컨볼루션 레이어 3
model.add(Conv2D(128, (3, 3), activation='relu')) 
model.add(MaxPooling2D((2, 2)))

# fully connected 레이어
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# 데이터를 NumPy 배열로 변환
x_train = np.array(x_train)
y_train = np.array(y_train)
x_val = np.array(x_val)
y_val = np.array(y_val)

# 모델 컴파일
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 모델 학습
model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=30, batch_size=16)

# 모델 저장
model.save('plastic_cup_classifier.h5')

Epoch 1/30


c:\Users\lavie\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 50ms/step - accuracy: 0.7252 - loss: 26.8032 - val_accuracy: 0.8861 - val_loss: 0.5423
Epoch 2/30
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 45ms/step - accuracy: 0.8743 - loss: 0.4610 - val_accuracy: 0.9620 - val_loss: 0.1088
Epoch 3/30
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 46ms/step - accuracy: 0.9829 - loss: 0.0911 - val_accuracy: 0.9747 - val_loss: 0.0751
Epoch 4/30
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 47ms/step - accuracy: 0.9678 - loss: 0.0990 - val_accuracy: 0.8734 - val_loss: 0.2620
Epoch 5/30
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 47ms/step - accuracy: 0.9182 - loss: 0.2468 - val_accuracy: 0.9367 - val_loss: 0.2023
Epoch 6/30
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 47ms/step - accuracy: 0.9250 - loss: 0.2839 - val_accuracy: 0.9494 - val_loss: 0.1696
Epoch 7/30
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 47ms/step - accuracy: 0.9411 - loss: 0.6318 - val_accuracy: 0.8228 - val_loss: 0.5360
Epoch 8/30
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 49ms/step - accuracy: 0.9139 - loss: 0.4609 - val_accuracy: 0.9241 - val_loss: 0

In [8]:
# 모델 불러오기
from tensorflow.keras.models import load_model
model = load_model('plastic_cup_classifier.h5')

# 테스트 데이터셋 준비
x_test = np.array(x_val)
y_test = np.array(y_val)

# 모델 평가
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=2)
print(f'Test accuracy: {test_acc * 100:.2f}%')

# 예측값 구하기
y_pred = (model.predict(x_test) > 0.5).astype(int)

# 정확도 계산
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy (using sklearn.metrics): {accuracy * 100:.2f}%')

3/3 - 0s - 66ms/step - accuracy: 0.9114 - loss: 0.2313
Test accuracy: 91.14%
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
Accuracy (using sklearn.metrics): 91.14%
